In [1]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import numpy as np
from pandas.api.types import is_numeric_dtype
from numpy import asarray
from sklearn.preprocessing import MinMaxScaler
sys.path.insert(0, '../scripts')
sys.path.insert(0,'../data')
from visualize import *


In [2]:
df = pd.read_csv("../data/cleaned_data.csv")

In [3]:
df.columns

Index(['Bearer Id', 'Start', 'Start ms', 'End', 'End ms', 'Dur. (ms)', 'IMSI',
       'MSISDN/Number', 'IMEI', 'Last Location Name', 'Avg RTT DL (ms)',
       'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)',
       'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)',
       'DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)',
       '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)',
       'UL TP < 10 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)',
       '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)',
       'HTTP DL (Bytes)', 'HTTP UL (Bytes)', 'Activity Duration DL (ms)',
       'Activity Duration UL (ms)', 'Dur. (ms).1', 'Handset Manufacturer',
       'Handset Type', 'Nb of sec with 125000B < Vol DL',
       'Nb of sec with 1250B < Vol UL < 6250B',
       'Nb of sec with 31250B < Vol DL < 125000B',
       'Nb of sec with 37500B < Vol UL',
       'Nb of sec with 6250B < Vol DL < 31250B',
       'Nb of sec with 6250B < Vol UL < 37500B',


user experience is related, most of the time, to network parameter performances or the customers’ device characteristics. Will try to analyze user's experience in this notebook.

Aggregated data per customer, on the following parameters:
- Average TCP retransmission
- Average RTT
- Handset type
- Average throughput


In [15]:
# Avg TCP retransmission per user
print("Top 10 users ranked based on Avg TCP retransmission")
df["Avg TCP retransmission(Bytes)"] = (df["TCP DL Retrans. Vol (Bytes)"]+df["TCP UL Retrans. Vol (Bytes)"])
tcp_retrans = df.groupby('MSISDN/Number')["Avg TCP retransmission(Bytes)"].sum().reset_index()
tcp_retrans.sort_values(by = "Avg TCP retransmission(Bytes)",ascending=False).head(10)

Top 10 users ranked based on Avg TCP retransmission


,MSISDN/Number,Avg TCP retransmission(Bytes)
13526,3.362632e+10,10614231.0
13180,3.362578e+10,10024551.5
6437,3.361489e+10,10024551.5
37052,3.365973e+10,9434872.0
92923,3.376054e+10,8845192.5
76363,3.367588e+10,8845192.5
65118,3.366716e+10,7665833.5
13994,3.362708e+10,7076154.0
1279,3.360452e+10,7076154.0
92577,3.376041e+10,7076154.0


In [17]:
# Avg RTT per user
print("Top 10 users ranked based on Avg RTT")
df["Avg RTT (ms)"] = (df["Avg RTT UL (ms)"]+df["Avg RTT DL (ms)"])
tcp_retrans = df.groupby('MSISDN/Number')["Avg RTT (ms)"].sum().reset_index()
tcp_retrans.sort_values(by = "Avg RTT (ms)",ascending=False).head(10)

Top 10 users ranked based on Avg RTT


,MSISDN/Number,Avg RTT (ms)
76363,3.367588e+10,891.0
6437,3.361489e+10,817.0
92923,3.376054e+10,811.0
13526,3.362632e+10,784.0
37052,3.365973e+10,778.0
666,3.360313e+10,728.0
92577,3.376041e+10,728.0
37470,3.365982e+10,723.0
13180,3.362578e+10,720.0
30225,3.365826e+10,707.0


In [10]:
# Handset Type per user count
print("Top Handset Types ranked by number of users")
handset = df.groupby("Handset Type")['MSISDN/Number'].count().reset_index()
handset = handset[handset["Handset Type"] != "undefined"]
handset.rename(columns={"MSISDN/Number":"user_count"},inplace=True)
handset.sort_values(by = "user_count",ascending=False).head(10)

Top Handset Types ranked by number of users


,Handset Type,user_count
316,Huawei B528S-23A,19727
53,Apple iPhone 6S (A1688),9413
49,Apple iPhone 6 (A1586),9012
59,Apple iPhone 7 (A1778),6304
73,Apple iPhone Se (A1723),5176
66,Apple iPhone 8 (A1905),4985
78,Apple iPhone Xr (A2105),4562
955,Samsung Galaxy S8 (Sm-G950F),4480
75,Apple iPhone X (A1901),3810
838,Samsung Galaxy A5 Sm-A520F,3708


In [18]:
#Avg throughput
print("Top 10 users ranked based on Avg throughput")
df["Avg throughput (kbps)"] = (df["Avg Bearer TP DL (kbps)"]+df["Avg Bearer TP UL (kbps)"])
avg_throughput = df.groupby('MSISDN/Number')["Avg throughput (kbps)"].sum().reset_index()
avg_throughput.sort_values(by = "Avg throughput (kbps)",ascending=False).head(10)

Top 10 users ranked based on Avg throughput


,MSISDN/Number,Avg throughput (kbps)
35436,3.365936e+10,200043.00
37470,3.365982e+10,198291.00
57241,3.366471e+10,197756.25
6437,3.361489e+10,192141.25
97584,3.376233e+10,186535.75
30225,3.365826e+10,175966.25
37052,3.365973e+10,168253.25
76363,3.367588e+10,167048.75
1279,3.360452e+10,166912.00
36257,3.365955e+10,166646.00


Top, bottom and most frequent:
TCP values in the dataset. 
RTT values in the dataset.
Throughput values in the dataset.


Task 3.3 - Compute & report:
The distribution of the average throughput  per handset type and provide interpretation for your findings.
The average TCP retransmission view per handset type and provide interpretation for your findings.
Task 3.4 - Using the experience metrics above, perform a k-means clustering (where k = 3) to segment users into groups of experiences and provide a brief description of each cluster. (The description must define each group based on your understanding of the data)